<a href="https://colab.research.google.com/github/greenpenguinkaa/greenpenguinkaa/blob/main/tgBot_football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install aiogram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.0/610.0 kB 10.6 MB/s eta 0:00:00


In [ ]:
# Установим библиотеку nest_asyncio
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!pip3 install aiosqlite

In [ ]:
import aiosqlite
import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram.utils.keyboard import InlineKeyboardBuilder, ReplyKeyboardBuilder
from aiogram import F

# Включаем логирование, чтобы не пропустить важные сообщения
logging.basicConfig(level=logging.INFO)

# Замените "YOUR_BOT_TOKEN" на токен, который вы получили от BotFather
API_TOKEN =  ""

# Объект бота
bot = Bot(token=API_TOKEN)
# Диспетчер
dp = Dispatcher()

# Зададим имя базы данных
DB_NAME = 'quiz_bot.db'

# DICT_DATA = 'data/qiuz_data.json'

# with open(DICT_DATA, 'r') as j:
#     quiz_data = json.loads(j.read)

quiz_data = [
    {
        'question': 'Самый титулованный клуб в лиге чемпионов?',
        'options': ['Реал Мадрид', 'Бавария Мюнхен', 'Манчестер Юнайтед', 'ПСЖ'],
        'correct_option': 0
    },
    {
        'question': 'Какой российский клуб первым выиграл еврокубок?',
        'options': ['Торпедо', 'ЦСКА', 'Динамо', 'Краснодар'],
        'correct_option': 1
    },
    {
        'question': 'Кто выиграл чемпионат Италии в сезоне 2023/2024?',
        'options': ['Милан', 'Фиоерентина', 'Рома', 'Интер'],
        'correct_option': 3
    },
    {
        'question': 'Какой клуб единственный в истории выиграл английский чемпионат без поражений?',
        'options': ['Челси', 'Асренал', 'Манчетсер Юнайтед', 'Ньюкасл Юнайтед'],
        'correct_option': 1
    },
    {
        'question': 'Кто выиграл чемпионат Англии в сезоне 2023/2024?',
        'options': ['Челси', 'Арсенал', 'Манчестер Сити', 'Манчестер Юнайтед'],
        'correct_option': 2
    },
    {
        'question': 'Кто выиграл чемпионат Фрвнции в сезоне 2023/2024?',
        'options': ['Лион', 'Марсель', 'Монако', 'ПСЖ'],
        'correct_option': 3
    },
    {
        'question': 'Кто выиграл чемпионат Фрвнции в сезоне 2023/2024?',
        'options': ['Лион', 'Марсель', 'Монако', 'ПСЖ'],
        'correct_option': 3
    },
    {
        'question': 'Как называют финал Лиги Чемпионов сезона 2004/2005?',
        'options': ['Стамбульское чудо', 'Миланское Чудо', 'Ливерпульское чудо', 'Марсельское чудо'],
        'correct_option': 0
    },
    {
        'question': 'Какой клуб больше всего выигрывал Кубков России?',
        'options': ['Локомотив', 'Ростов', 'Урал', 'Динамо'],
        'correct_option': 0
    },
    {
        'question': 'Кто выиграл чемпионат Фрвнции в сезоне 2023/2024?',
        'options': ['Лион', 'Марсель', 'Монако', 'ПСЖ'],
        'correct_option': 3
    },
]



def generate_options_keyboard(answer_options, right_answer):
    builder = InlineKeyboardBuilder()

    for option in answer_options:
        builder.add(types.InlineKeyboardButton(
            text=option,
            callback_data="right_answer" if option == right_answer else "wrong_answer")
        )

    builder.adjust(1)
    return builder.as_markup()


@dp.callback_query(F.data == "right_answer")
async def right_answer(callback: types.CallbackQuery):
    await callback.bot.edit_message_reply_markup(
        chat_id=callback.from_user.id,
        message_id=callback.message.message_id,
        reply_markup=None
    )

    await callback.message.answer("Верно!")
    current_question_index = await get_quiz_index(callback.from_user.id)
    current_score = await get_user_score(callback.from_user.id)
    current_question_index += 1
    current_score += 1
    await update_quiz_index(callback.from_user.id, current_question_index)
    await update_user_score(callback.from_user.id, current_score)


    if current_question_index < len(quiz_data):
        await get_question(callback.message, callback.from_user.id)
    else:
        await callback.message.answer(f"Это был последний вопрос. Квиз завершен!\nВаш результат: {current_score} правильных ответов")


@dp.callback_query(F.data == "wrong_answer")
async def wrong_answer(callback: types.CallbackQuery):
    await callback.bot.edit_message_reply_markup(
        chat_id=callback.from_user.id,
        message_id=callback.message.message_id,
        reply_markup=None
    )

    # Получение текущего вопроса из словаря состояний пользователя
    current_question_index = await get_quiz_index(callback.from_user.id)
    current_score = await get_user_score(callback.from_user.id)
    correct_option = quiz_data[current_question_index]['correct_option']
    await callback.message.answer(f"Неправильно. Правильный ответ: {quiz_data[current_question_index]['options'][correct_option]}")

    # Обновление номера текущего вопроса в базе данных
    current_question_index += 1
    await update_quiz_index(callback.from_user.id, current_question_index)
    await update_user_score(callback.from_user.id, current_score)


    if current_question_index < len(quiz_data):
        await get_question(callback.message, callback.from_user.id)
    else:
        await callback.message.answer(f"Это был последний вопрос. Квиз завершен!\nВаш результат: {current_score} правильных ответов")


# Хэндлер на команду /start
@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    builder = ReplyKeyboardBuilder()
    builder.add(types.KeyboardButton(text="Начать игру"))
    await message.answer("Добро пожаловать в квиз!", reply_markup=builder.as_markup(resize_keyboard=True))


async def get_question(message, user_id):
    # Получение текущего вопроса из словаря состояний пользователя
    current_question_index = await get_quiz_index(user_id)
    correct_index = quiz_data[current_question_index]['correct_option']
    opts = quiz_data[current_question_index]['options']
    kb = generate_options_keyboard(opts, opts[correct_index])
    await message.answer(f"{quiz_data[current_question_index]['question']}", reply_markup=kb)


async def new_quiz(message):
    user_id = message.from_user.id
    current_question_index = 0
    new_score = 0
    await update_quiz_index(user_id, current_question_index)
    await update_user_score(user_id, new_score)
    await get_question(message, user_id)


async def get_quiz_index(user_id):
     # Подключаемся к базе данных
     async with aiosqlite.connect(DB_NAME) as db:
        # Получаем запись для заданного пользователя
        async with db.execute('SELECT question_index FROM quiz_state WHERE user_id = (?)', (user_id, )) as cursor:
            # Возвращаем результат
            results = await cursor.fetchone()
            if results is not None:
                return results[0]
            else:
                return 0

async def get_user_score(user_id):
  async with aiosqlite.connect(DB_NAME) as db:
    async with db.execute('SELECT score FROM users WHERE user_id = ?', (user_id,)) as cursor:
      results = await cursor.fetchone()
      if results is not None:
        return results[0]
      else:
        return 0

async def update_quiz_index(user_id, index):
    # Создаем соединение с базой данных (если она не существует, она будет создана)
    async with aiosqlite.connect(DB_NAME) as db:
        # Вставляем новую запись или заменяем ее, если с данным user_id уже существует
        await db.execute('INSERT OR REPLACE INTO quiz_state (user_id, question_index) VALUES (?, ?)', (user_id, index))
        # Сохраняем изменения
        await db.commit()

async def update_user_score(user_id, new_score):
  async with aiosqlite.connect(DB_NAME) as db:
    await db.execute('INSERT INTO users (user_id, score) VALUES(?,?) ON CONFLICT(user_id) DO UPDATE SET score = excluded.score', (user_id, new_score))
    await db.commit()

# Хэндлер на команду /quiz
@dp.message(F.text=="Начать игру")
@dp.message(Command("quiz"))
async def cmd_quiz(message: types.Message):
    await message.answer(f"Давайте начнем квиз!")
    await new_quiz(message)



async def create_table():
    # Создаем соединение с базой данных (если она не существует, она будет создана)
    async with aiosqlite.connect(DB_NAME) as db:
        # Создаем таблицу
        await db.execute('''CREATE TABLE IF NOT EXISTS quiz_state (user_id INTEGER PRIMARY KEY, question_index INTEGER)''')
        await db.execute('''CREATE TABLE IF NOT EXISTS users (user_id INTEGER PRIMARY KEY, score INTEGER)''')
        # Сохраняем изменения
        await db.commit()

@dp.message(Command("help"))
async def cmd_start(message: types.Message):
  await message.answer(f"Команды бота:\n\start - начать взаимодействие с ботом\n\help - открыть помощь\n\quiz - Давайте начнем квиз!")


# Запуск процесса поллинга новых апдейтов
async def main():

    # Запускаем создание таблицы базы данных
    await create_table()

    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())